# NBA Project: Webscraping & Cleaning

# 1.1 Imports


In [1]:
from urllib.request import urlopen
from bs4 import BeautifulSoup

import pandas as pd
import numpy as np

# 1.2 Webscraping

Using the Python library BeautifulSoup, I scraped basketball-reference.com to retrieve player statistics for each season from 2017-2020 (4 seasons). For each of these seasons, I retrieved 2 datasets which I will eventually merge together. I grabbed Season Totals, and Season Advanced Statistics. Below are a few features included in each dataset that will eventually be merged together.
    
Season Totals: 
* Points
* Field Goal %
* Rebounds
* Assists
* Turnovers
   
Advanced: 
* PER - Player Efficency Rating
* TS% - True Shooting Percentage
* OWS, DWS, WS - Win Shares (offensive & defensive)

In [2]:
# grabs stat totals by year from basketball reference and returns a pandas df. Drops the GS col
def nba_totals_by_year(year):
    url = 'https://www.basketball-reference.com/leagues/NBA_{}_totals.html'.format(year)
    html = urlopen(url)
    soup = BeautifulSoup(html)
    # finds rows, then headers
    soup.findAll('tr', limit=2)
    headers = [th.getText() for th in soup.findAll('tr', limit=2)[0].findAll('th')]
    headers = headers[1:]
    rows = soup.findAll('tr')[1:]
    # for each row, grab the text 
    player_stats = [[td.getText() for td in rows[i].findAll('td')] for i in range(len(rows))]
    # drops GS col
    return pd.DataFrame(player_stats, columns=headers).drop(columns='GS')

In [3]:
# grabs only the advanced stats that we want from each year (PER, TS%, OWS, DWS, WS)
def nba_adv_stats_by_year(year):
    url = 'https://www.basketball-reference.com/leagues/NBA_{}_advanced.html'.format(year)
    html = urlopen(url)
    soup = BeautifulSoup(html)
    headers = [th.getText() for th in soup.findAll('tr', limit=2)[0].findAll('th')]
    headers = headers[1:]
    rows = soup.findAll('tr')[1:]
    adv_stats = [[td.getText() for td in rows[i].findAll('td')] for i in range(len(rows))]
    df = pd.DataFrame(adv_stats, columns=headers)
    df = df[['Player','Tm', 'PER', 'TS%', 'OWS', 'DWS', 'WS']]
    return df 

In [4]:
# totals for last 4 seasons
nba_totals_2020 = nba_totals_by_year(2020)
nba_totals_2019 = nba_totals_by_year(2019)
nba_totals_2018 = nba_totals_by_year(2018)
nba_totals_2017 = nba_totals_by_year(2017)

In [5]:
# adv stats for last 4 seasons 
nba_adv_2020 = nba_adv_stats_by_year(2020)
nba_adv_2019 = nba_adv_stats_by_year(2019)
nba_adv_2018 = nba_adv_stats_by_year(2018)
nba_adv_2017 = nba_adv_stats_by_year(2017)

# 1.3 Data Cleaning

The initial shape of the 2020 totals dataset was 677 rows and 28 columns. The 2020 advanced statistics dataset holds the same amount of rows and only the 5 columns we need. 

I preview a sample of each of the datasets below. 

From the info() function, we know there are 651 non-null values which means there will be null values in our dataset we'll have to deal with.

In [6]:
print('2020 totals shape ... ', nba_totals_2020.shape)
print('2020 adv shape ... ', nba_adv_2020.shape)

2020 totals shape ...  (677, 28)
2020 adv shape ...  (677, 7)


The cell below displays a sample of of the 2020 Season Totals dataframe

In [7]:
display(nba_totals_2020.sample(5))

,Player,Pos,Age,Tm,G,MP,FG,FGA,FG%,3P,...,FT%,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS
151,Robert Covington,PF,29,MIN,48,1411,216,496,.435,108,...,.798,36,251,287,56,80,45,73,148,615
243,Treveon Graham,SF,26,TOT,55,929,87,242,.360,33,...,.643,51,99,150,45,24,6,29,102,243
156,Torrey Craig,SF,29,DEN,58,1072,124,269,.461,45,...,.611,63,126,189,47,23,36,25,135,315
416,Luc Mbah a Moute,PF,33,HOU,3,25,2,5,.400,0,...,.500,1,1,2,0,2,0,1,0,5
107,Devontae Cacok,C,23,LAL,1,9,3,6,.500,0,...,,2,3,5,1,0,0,0,0,6


The cell below displays a sample of the 2020 Season Advanced Statistics dataframe

In [8]:
display(nba_adv_2020.sample(5))

,Player,Tm,PER,TS%,OWS,DWS,WS
75,Tony Bradley,UTA,21.7,.681,2.2,1.1,3.3
40,Darius Bazley,OKC,9.6,.497,-0.5,1.5,1.0
658,Lou Williams,LAC,17.3,.546,2.3,1.8,4.1
562,Dennis Schröder,OKC,16.7,.575,1.9,1.8,3.6
203,Jacob Evans,TOT,5.5,.436,-0.7,0.2,-0.5


In [9]:
nba_totals_2020.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 677 entries, 0 to 676
Data columns (total 28 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Player  651 non-null    object
 1   Pos     651 non-null    object
 2   Age     651 non-null    object
 3   Tm      651 non-null    object
 4   G       651 non-null    object
 5   MP      651 non-null    object
 6   FG      651 non-null    object
 7   FGA     651 non-null    object
 8   FG%     651 non-null    object
 9   3P      651 non-null    object
 10  3PA     651 non-null    object
 11  3P%     651 non-null    object
 12  2P      651 non-null    object
 13  2PA     651 non-null    object
 14  2P%     651 non-null    object
 15  eFG%    651 non-null    object
 16  FT      651 non-null    object
 17  FTA     651 non-null    object
 18  FT%     651 non-null    object
 19  ORB     651 non-null    object
 20  DRB     651 non-null    object
 21  TRB     651 non-null    object
 22  AST     651 non-null    ob

## 1.3.1 Dealing with players on multiple teams

Above, we found out that there are 677 rows in the 2020 Totals Dataset. From the website where I originally pulled the data, I know there are only 529 players from the 2020 season. I only want one row for each player for each season, so we want to end up with 529 rows for 2020. 

The issue is there are multiple rows for players that played for multiple teams in one season. For these players, they have a row for each team they played for, holding only stats for games with that team in addition to a row holding the combined totals for that player across all teams which they played for that season. I want to keep the row with total stats, and change the team name for that row from 'TOT' to the team they played the most games with that season. In future analysis, we may want to examine players by team, and we dont want players to not have a team.

The cell below shows 5 players that played for multiple teams, which is why their Team ('Tm') is 'TOT', which stands for Total.

In [10]:
display(nba_totals_2020[nba_totals_2020['Tm'] == 'TOT'].sample(5))

,Player,Pos,Age,Tm,G,MP,FG,FGA,FG%,3P,...,FT%,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS
358,Michael Kidd-Gilchrist,PF,26,TOT,25,281,22,66,.333,5,...,.786,14,53,67,14,2,6,16,37,60
224,Wenyen Gabriel,PF,22,TOT,30,235,21,48,.438,6,...,.682,21,31,52,8,11,7,16,39,63
384,Alex Len,C,26,TOT,55,970,177,319,.555,16,...,.648,100,221,321,50,23,48,55,127,438
652,Andrew Wiggins,SF,24,TOT,54,1858,437,978,.447,111,...,.709,66,208,274,198,45,53,132,127,1175
282,John Henson,C,29,TOT,40,600,95,171,.556,8,...,.500,46,116,162,54,24,42,36,78,221


To fix this problem, I wrote a function that as input takes a dataframe, and returns a new dataframe with only one row per player, the row with total statistics for that year. Then  it renames the 'Tm' value to the team the player appeared in the most games with (instead of 'TOT').

Later on in the project, I want to be able to have a team for each player and all of their statistics for the season on one row versus multiple rows. 

In [11]:
def drop_rows_and_rename(df):
    df = df.copy()
    df_without_TOT = df[df['Tm'] != 'TOT']
    df_without_TOT = df_without_TOT.groupby(['Player']).max().reset_index()
    player_teams = df_without_TOT[['Player','Tm']]    
    #dict that will be converted to the final df
    player_team_dict = dict(zip(player_teams.Player, player_teams.Tm)) 
    # in the Team col, mapped the player_team_dict and filled na with Team
    df['Tm'] = df['Player'].map(player_team_dict).fillna(df['Tm'])
    df = df.drop_duplicates('Player', keep='first').reset_index().drop('index', axis=1)
    return df

In [12]:
# drops unnecessary rows and renames 'TOT' team players for each season
nba_totals_2020 = drop_rows_and_rename(nba_totals_2020)
nba_totals_2019 = drop_rows_and_rename(nba_totals_2019)
nba_totals_2018 = drop_rows_and_rename(nba_totals_2018)
nba_totals_2017 = drop_rows_and_rename(nba_totals_2017)

In [13]:
nba_adv_2020 = drop_rows_and_rename(nba_adv_2020)
nba_adv_2019 = drop_rows_and_rename(nba_adv_2019)
nba_adv_2018 = drop_rows_and_rename(nba_adv_2018)
nba_adv_2017 = drop_rows_and_rename(nba_adv_2017)

In [14]:
display(nba_totals_2020.sample(5))
display(nba_totals_2019.head())

,Player,Pos,Age,Tm,G,MP,FG,FGA,FG%,3P,...,FT%,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS
126,Dewayne Dedmon,C,30,SAC,44,774,102,255,.400,22,...,.833,60,189,249,22,25,41,57,133,256
165,Wenyen Gabriel,PF,22,SAC,30,235,21,48,.438,6,...,.682,21,31,52,8,11,7,16,39,63
405,Shamorie Ponds,PG,21,TOR,4,11,3,5,.600,1,...,1.000,0,1,1,2,0,1,0,2,9
412,Dwight Powell,C,28,DAL,40,1061,146,229,.638,10,...,.667,75,152,227,59,34,22,36,102,376
205,Shaquille Harrison,SG,26,CHI,43,484,77,165,.467,16,...,.780,22,64,86,49,34,19,19,55,209


,Player,Pos,Age,Tm,G,MP,FG,FGA,FG%,3P,...,FT%,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS
0,Álex Abrines,SG,25,OKC,31,588,56,157,.357,41,...,.923,5,43,48,20,17,6,14,53,165
1,Quincy Acy,PF,28,PHO,10,123,4,18,.222,2,...,.700,3,22,25,8,1,4,4,24,17
2,Jaylen Adams,PG,22,ATL,34,428,38,110,.345,25,...,.778,11,49,60,65,14,5,28,45,108
3,Steven Adams,C,25,OKC,80,2669,481,809,.595,0,...,.500,391,369,760,124,117,76,135,204,1108
4,Bam Adebayo,C,21,MIA,82,1913,280,486,.576,3,...,.735,165,432,597,184,71,65,121,203,729


We can see above after the function is applied there are no more 'TOT' teams and each player has their own unique row. 

In the original dataset extracted, Trevor Ariza was listed as a player in 3 different rows. The cell below shows the only row with Trevor Ariza in the new dataset, and because he appeared in the most games that season with Sacramento, the value for 'Tm' is 'SAC'.

In [15]:
display(drop_rows_and_rename(nba_totals_2020[nba_totals_2020['Player'] == 'Trevor Ariza']))

,Player,Pos,Age,Tm,G,MP,FG,FGA,FG%,3P,...,FT%,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS
0,Trevor Ariza,SF,34,SAC,53,1493,142,324,.438,77,...,.838,34,212,246,91,69,15,56,111,423


The code cell below counts the number of times each unique value for 'Player' appears in the dataset. We want one unique row for each individual player, so the value count for each player should be 1, and the total length should be 529. 

In [16]:
# num of times each player shows up in the 2020 Totals dataframe
nba_2020_counts = nba_totals_2020['Player'].value_counts() 
print(nba_2020_counts)

Kadeem Allen            1
Dwight Powell           1
Boban Marjanović        1
Brandon Goodwin         1
Kostas Antetokounmpo    1
                       ..
Jaren Jackson Jr.       1
Jonah Bolden            1
Devin Booker            1
Willie Cauley-Stein     1
Doug McDermott          1
Name: Player, Length: 529, dtype: int64


# 1.3.2 Merging advanced and total stats for each season

Now that each season has one row for each player we can merge the advanced statistics with the total statistics into one dataframe per season, and eventually we will merge each of these seasons together into one dataframe.

I also drop the second 'Player' and 'Tm' columns so there aren't any duplicates in our final df's.

Lastly I add a new column to each season's dataframe that gives the year so in the final dataframe we can determine one season from another.

In [17]:
# merging totals with advanced stats
nba_2020 = pd.concat([nba_totals_2020, nba_adv_2020], axis=1).reindex(nba_totals_2020.index)
nba_2019 = pd.concat([nba_totals_2019, nba_adv_2019], axis=1).reindex(nba_totals_2019.index)
nba_2018 = pd.concat([nba_totals_2018, nba_adv_2018], axis=1).reindex(nba_totals_2018.index)
nba_2017 = pd.concat([nba_totals_2017, nba_adv_2017], axis=1).reindex(nba_totals_2017.index)

In [18]:
# eliminates duplicated columns
nba_2020 = nba_2020.loc[:, ~nba_2020.columns.duplicated()]
nba_2019 = nba_2019.loc[:, ~nba_2019.columns.duplicated()]
nba_2018 = nba_2018.loc[:, ~nba_2018.columns.duplicated()]
nba_2017 = nba_2017.loc[:, ~nba_2017.columns.duplicated()]

In [19]:
# makes a new column that gives the season for each dataframe, before we merge them all
nba_2020['Season'] = 2020
nba_2019['Season'] = 2019
nba_2018['Season'] = 2018
nba_2017['Season'] = 2017

In [20]:
display(nba_2020.head())

,Player,Pos,Age,Tm,G,MP,FG,FGA,FG%,3P,...,BLK,TOV,PF,PTS,PER,TS%,OWS,DWS,WS,Season
0,Steven Adams,C,26,OKC,63,1680,283,478,.592,1,...,67,94,122,684,20.5,.604,3.8,2.7,6.5,2020
1,Bam Adebayo,PF,22,MIA,72,2417,440,790,.557,2,...,93,204,182,1146,20.3,.598,4.6,3.9,8.5,2020
2,LaMarcus Aldridge,C,34,SAS,53,1754,391,793,.493,61,...,87,74,128,1001,19.7,.571,3.0,1.4,4.5,2020
3,Kyle Alexander,C,23,MIA,2,13,1,2,.500,0,...,0,1,1,2,4.7,.500,0.0,0.0,0.0,2020
4,Nickeil Alexander-Walker,SG,21,NOP,47,591,98,266,.368,46,...,8,54,57,267,8.9,.473,-0.7,0.4,-0.2,2020


# 1.3.3 Merging each season into one dataframe

We can now append all four seasons into one dataframe, and then sort them by Player, and then Season. The dataframe will start with players who's names start with the letter A, and each Player's rows will be sorted from their earliest season on top to their most recent seasons on bottom.

In [21]:
merged_df = pd.concat([nba_2020, nba_2019, nba_2018, nba_2017], axis=0)
merged_df = merged_df.sort_values(['Player','Season'], ascending=[True, True])

In [22]:
display(merged_df.head(20))

,Player,Pos,Age,Tm,G,MP,FG,FGA,FG%,3P,...,BLK,TOV,PF,PTS,PER,TS%,OWS,DWS,WS,Season
170,A.J. Hammons,C,24,DAL,22,163,17,42,.405,5,...,13,10,21,48,8.4,.472,-0.2,0.2,0.0,2017
58,Aaron Brooks,PG,32,IND,65,894,121,300,.403,48,...,9,66,93,322,9.5,.507,-0.2,0.5,0.3,2017
65,Aaron Brooks,PG,33,MIN,32,189,28,69,.406,11,...,0,11,28,75,9.8,.508,0.1,0.1,0.1,2018
157,Aaron Gordon,SF,21,ORL,80,2298,393,865,.454,77,...,40,89,172,1019,14.5,.530,2.0,1.7,3.7,2017
179,Aaron Gordon,PF,22,ORL,58,1909,375,865,.434,115,...,45,107,111,1022,16.5,.530,0.9,2.0,2.9,2018
189,Aaron Gordon,PF,23,ORL,78,2633,470,1046,.449,121,...,56,162,172,1246,15.1,.538,1.8,3.3,5.1,2019
178,Aaron Gordon,PF,24,ORL,62,2017,335,767,.437,73,...,39,100,125,894,15.1,.516,1.4,2.3,3.7,2020
181,Aaron Harrison,SG,22,CHO,5,17,0,4,.000,0,...,0,0,2,1,-2.2,.102,-0.1,0.0,-0.1,2017
201,Aaron Harrison,SG,23,DAL,9,233,19,69,.275,9,...,2,3,27,60,5.1,.392,-0.3,0.2,-0.1,2018
229,Aaron Holiday,PG,22,IND,50,646,105,262,.401,43,...,13,40,71,294,11.9,.518,0.1,0.8,0.9,2019


For our merged dataframe, we start with 2089 rows and 34 columns

In [23]:
print(merged_df.shape)

(2089, 34)


# 1.3.4 Changing columns to correct data types

In order to do some exploratory analysis of our data, we have to change the data types of our numeric columns, as they are currently set as string data types. When I execute the cell below which sorts all of our rows by highest point scorerers, I would expect different players to lead the league. But because they are currently stored as strings, I only get players with points up to 1000. This is because they are being as read as a string, and not as numbers. 

In [24]:
display(merged_df.sort_values(by='PTS', ascending=False).head(5))

,Player,Pos,Age,Tm,G,MP,FG,FGA,FG%,3P,...,BLK,TOV,PF,PTS,PER,TS%,OWS,DWS,WS,Season
176,Taj Gibson,PF,32,MIN,82,2726,425,737,.577,7,...,59,89,218,999,15.4,.610,5.4,1.7,7.2,2018
95,Jae Crowder,SF,26,BOS,72,2335,333,720,.463,157,...,23,80,161,999,14.9,.613,4.1,2.5,6.7,2017
203,Joe Harris,SF,28,BRK,69,2123,381,784,.486,172,...,17,104,159,998,13.3,.606,2.5,1.8,4.3,2020
193,Tim Hardaway Jr.,SG,25,NYK,57,1885,360,856,.421,130,...,10,92,123,996,14.4,.533,1.6,1.1,2.6,2018
246,Joe Ingles,PF,31,UTA,82,2568,359,802,.448,189,...,20,193,180,994,13.4,.581,2.5,3.6,6.0,2019


You would expect LeBron James to be at the top of our list of point scorers, he scored well over 1000 points in each season he played in, and was still not included in our previous cell of code that produced our top 5 scorers in a season. 

In [25]:
display(merged_df[merged_df['Player'] == 'LeBron James'])

,Player,Pos,Age,Tm,G,MP,FG,FGA,FG%,3P,...,BLK,TOV,PF,PTS,PER,TS%,OWS,DWS,WS,Season
220,LeBron James,SF,32,CLE,74,2794,736,1344,.548,124,...,44,303,134,1954,27.0,.619,9.8,3.0,12.9,2017
248,LeBron James,PF,33,CLE,82,3026,857,1580,.542,149,...,71,347,136,2251,28.6,.621,11.0,3.0,14.0,2018
258,LeBron James,SF,34,LAL,55,1937,558,1095,.510,111,...,33,197,94,1505,25.6,.588,4.7,2.6,7.2,2019
250,LeBron James,PG,35,LAL,67,2316,643,1303,.493,148,...,36,261,118,1698,25.5,.577,6.2,3.6,9.8,2020


To deal with this, we change the data types to numeric values so we can perform further analysis of our data. The function loops over each column in our df that is not in the string columns list (str_cols) and changes their dtype to numeric values.  

In [26]:
# the only columns we want to remain as string data types
str_cols = ['Player', 'Pos', 'Tm']
def cols_to_numeric(df):
    df.copy()
    numeric_cols = [i for i in df.columns if i not in str_cols]
    for col in numeric_cols:
        df[col]=pd.to_numeric(df[col])
    return df

In [27]:
merged_df = cols_to_numeric(merged_df)

After applying the cols_to_numeric function, we print out the data types of each column to check if they are what we expect them to be changed to. The only string columns should be the 'Player', 'Pos', and 'Tm' columns which will be listed as 'object' dtypes. 

In [28]:
print(merged_df.dtypes)

Player     object
Pos        object
Age       float64
Tm         object
G         float64
MP        float64
FG        float64
FGA       float64
FG%       float64
3P        float64
3PA       float64
3P%       float64
2P        float64
2PA       float64
2P%       float64
eFG%      float64
FT        float64
FTA       float64
FT%       float64
ORB       float64
DRB       float64
TRB       float64
AST       float64
STL       float64
BLK       float64
TOV       float64
PF        float64
PTS       float64
PER       float64
TS%       float64
OWS       float64
DWS       float64
WS        float64
Season      int64
dtype: object


# 1.3.4 Dealing with null rows & values

We want to figure out where we have null rows and values. First, we look to find all rows with null values in the 'Player' column

In [29]:
null_players = merged_df[merged_df['Player'].isna()]

In [30]:
display(null_players)

,Player,Pos,Age,Tm,G,MP,FG,FGA,FG%,3P,...,BLK,TOV,PF,PTS,PER,TS%,OWS,DWS,WS,Season
20,None,None,NaN,None,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2017
20,None,None,NaN,None,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2018
20,None,None,NaN,None,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2019
20,None,None,NaN,None,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2020


It seems we have 4 null rows that were likely added to the dataframe during the webscrape, as they all were on row index 20 within each season.

The cell below drops rows from our data where the row index is 20, handling our null row case from above.

In [31]:
# drop null rows, which are row index 20
merged_df = merged_df.drop(20).reset_index(drop=True)

The cell below should produce zero rows, meaning there are no rows where the Player column is null.

In [32]:
display(merged_df[merged_df['Player'].isna()])

,Player,Pos,Age,Tm,G,MP,FG,FGA,FG%,3P,...,BLK,TOV,PF,PTS,PER,TS%,OWS,DWS,WS,Season


Now, we want to see which columns still have null values and find the best way to handle them. 

In [33]:
# put comment here to explain code
nan_vals = merged_df.isna()
nan_cols = nan_vals.any()
cols_with_nan = merged_df.columns[nan_cols].to_list()

null_rows_only = merged_df[merged_df.isna().any(axis=1)]

In [34]:
print(cols_with_nan)

['FG%', '3P%', '2P%', 'eFG%', 'FT%', 'TS%']


The only columns with null values are percentage based statistics, which gives us a hint as to what the issue may be. Let's check to see how many shot attempts these players with null values in those columns took. 

In [35]:
cols_with_nan += ['3PA', 'FTA']

In [36]:
display(null_rows_only.loc[:,cols_with_nan])

,FG%,3P%,2P%,eFG%,FT%,TS%,3PA,FTA
31,0.389,NaN,0.389,0.389,0.667,0.455,0.0,9.0
47,0.423,0.364,0.467,0.500,NaN,0.500,11.0,0.0
62,0.625,NaN,0.625,0.625,0.875,0.692,0.0,8.0
63,0.357,NaN,0.357,0.357,0.357,0.372,0.0,14.0
68,0.357,NaN,0.357,0.357,0.727,0.478,0.0,11.0
...,...,...,...,...,...,...,...,...
2076,1.000,NaN,1.000,1.000,NaN,1.000,0.0,0.0
2078,0.667,0.000,0.750,0.667,NaN,0.667,1.0,0.0
2082,0.200,NaN,0.200,0.200,0.500,0.255,0.0,2.0
2083,0.477,NaN,0.477,0.477,0.590,0.517,0.0,39.0


We see in the cell above, for each row with atleast one null value, the player will have either 0 FTA or 0 3PA. For example, if a player didnt take a 3 point attempt all season they're going to have a null (empty) value for their 3P% statistic. 

We need to change these null values to 0 to clean the dataset of all null values for future analysis. 

In [37]:
final_df = merged_df.fillna(0.0)

In [38]:
display(final_df.head(10))

,Player,Pos,Age,Tm,G,MP,FG,FGA,FG%,3P,...,BLK,TOV,PF,PTS,PER,TS%,OWS,DWS,WS,Season
0,A.J. Hammons,C,24.0,DAL,22.0,163.0,17.0,42.0,0.405,5.0,...,13.0,10.0,21.0,48.0,8.4,0.472,-0.2,0.2,0.0,2017
1,Aaron Brooks,PG,32.0,IND,65.0,894.0,121.0,300.0,0.403,48.0,...,9.0,66.0,93.0,322.0,9.5,0.507,-0.2,0.5,0.3,2017
2,Aaron Brooks,PG,33.0,MIN,32.0,189.0,28.0,69.0,0.406,11.0,...,0.0,11.0,28.0,75.0,9.8,0.508,0.1,0.1,0.1,2018
3,Aaron Gordon,SF,21.0,ORL,80.0,2298.0,393.0,865.0,0.454,77.0,...,40.0,89.0,172.0,1019.0,14.5,0.530,2.0,1.7,3.7,2017
4,Aaron Gordon,PF,22.0,ORL,58.0,1909.0,375.0,865.0,0.434,115.0,...,45.0,107.0,111.0,1022.0,16.5,0.530,0.9,2.0,2.9,2018
5,Aaron Gordon,PF,23.0,ORL,78.0,2633.0,470.0,1046.0,0.449,121.0,...,56.0,162.0,172.0,1246.0,15.1,0.538,1.8,3.3,5.1,2019
6,Aaron Gordon,PF,24.0,ORL,62.0,2017.0,335.0,767.0,0.437,73.0,...,39.0,100.0,125.0,894.0,15.1,0.516,1.4,2.3,3.7,2020
7,Aaron Harrison,SG,22.0,CHO,5.0,17.0,0.0,4.0,0.000,0.0,...,0.0,0.0,2.0,1.0,-2.2,0.102,-0.1,0.0,-0.1,2017
8,Aaron Harrison,SG,23.0,DAL,9.0,233.0,19.0,69.0,0.275,9.0,...,2.0,3.0,27.0,60.0,5.1,0.392,-0.3,0.2,-0.1,2018
9,Aaron Holiday,PG,22.0,IND,50.0,646.0,105.0,262.0,0.401,43.0,...,13.0,40.0,71.0,294.0,11.9,0.518,0.1,0.8,0.9,2019


The cell below displays any rows with atleast one null value. There should be 0 rows.

In [39]:
display(final_df[final_df.isna().any(axis=1)])

,Player,Pos,Age,Tm,G,MP,FG,FGA,FG%,3P,...,BLK,TOV,PF,PTS,PER,TS%,OWS,DWS,WS,Season


# 1.4 Structured dataset to CSV
 
Now that our dataset is structured to the format we can work with, we export it to CSV so it will be easier to access in future nb's.

In [40]:
# first 20 rows of final dataframe
display(final_df.head(20))

,Player,Pos,Age,Tm,G,MP,FG,FGA,FG%,3P,...,BLK,TOV,PF,PTS,PER,TS%,OWS,DWS,WS,Season
0,A.J. Hammons,C,24.0,DAL,22.0,163.0,17.0,42.0,0.405,5.0,...,13.0,10.0,21.0,48.0,8.4,0.472,-0.2,0.2,0.0,2017
1,Aaron Brooks,PG,32.0,IND,65.0,894.0,121.0,300.0,0.403,48.0,...,9.0,66.0,93.0,322.0,9.5,0.507,-0.2,0.5,0.3,2017
2,Aaron Brooks,PG,33.0,MIN,32.0,189.0,28.0,69.0,0.406,11.0,...,0.0,11.0,28.0,75.0,9.8,0.508,0.1,0.1,0.1,2018
3,Aaron Gordon,SF,21.0,ORL,80.0,2298.0,393.0,865.0,0.454,77.0,...,40.0,89.0,172.0,1019.0,14.5,0.530,2.0,1.7,3.7,2017
4,Aaron Gordon,PF,22.0,ORL,58.0,1909.0,375.0,865.0,0.434,115.0,...,45.0,107.0,111.0,1022.0,16.5,0.530,0.9,2.0,2.9,2018
5,Aaron Gordon,PF,23.0,ORL,78.0,2633.0,470.0,1046.0,0.449,121.0,...,56.0,162.0,172.0,1246.0,15.1,0.538,1.8,3.3,5.1,2019
6,Aaron Gordon,PF,24.0,ORL,62.0,2017.0,335.0,767.0,0.437,73.0,...,39.0,100.0,125.0,894.0,15.1,0.516,1.4,2.3,3.7,2020
7,Aaron Harrison,SG,22.0,CHO,5.0,17.0,0.0,4.0,0.000,0.0,...,0.0,0.0,2.0,1.0,-2.2,0.102,-0.1,0.0,-0.1,2017
8,Aaron Harrison,SG,23.0,DAL,9.0,233.0,19.0,69.0,0.275,9.0,...,2.0,3.0,27.0,60.0,5.1,0.392,-0.3,0.2,-0.1,2018
9,Aaron Holiday,PG,22.0,IND,50.0,646.0,105.0,262.0,0.401,43.0,...,13.0,40.0,71.0,294.0,11.9,0.518,0.1,0.8,0.9,2019


In [41]:
final_df.to_csv('nba_player_stats_17_20.csv')